In [25]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from itertools import combinations
from collections import Counter
#from ampligraph.latent_features import ConvE, DistMult, save_model, restore_model
#from ampligraph.discovery import discover_facts, find_nearest_neighbours, query_topn
from pyvis.network import Network
import matplotlib.pyplot as plt


In [7]:
pip install pyvis

     ------------------------------------ 748.9/748.9 kB 397.4 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for pyvis: started
  Running setup.py install for pyvis: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: pyvis is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559


In [66]:
q = pd.read_json('part_3_clean.json')
articles = pd.DataFrame()
articles['_id'] = q['_id']
articles['references'] = q['references']
articles['authors'] = q['authors']
articles['year'] = q['year']

In [67]:
min_year = min(articles['year'])
max_year = max(articles['year'])

In [68]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 141155 entries, 0 to 141154
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   _id         141155 non-null  object
 1   references  141155 non-null  object
 2   authors     141155 non-null  object
 3   year        141155 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 5.4+ MB


In [69]:
articles = articles[0:200]

In [70]:
articles.head()

,_id,references,authors,year
0,56d85787dabfae2eee3492a5,"[55323d3f45cec66b6f9dde71, 5736954c6e3b12023e4...","[{'_id': '54068615dabfae8faa60c7c4', 'name': '...",2015
1,56d8578edabfae2eee34c50a,"[53e9990cb7602d9702146f11, 53e9a62eb7602d9702f...","[{'_id': '562d1b2045cedb3398d54413', 'name': '...",2015
2,56d85790dabfae2eee34d0b3,"[53e9a2ecb7602d9702bf5e30, 53e99f42b7602d97028...","[{'_id': '5606653745cedb3396868235', 'name': '...",2013
3,56d85791dabfae2eee34d7d2,"[53e9a97bb7602d97032d454a, 53e99fd6b7602d97028...","[{'_id': '53f44c92dabfaefedbb2c18b', 'name': '...",2015
4,56d85796dabfae2eee35068d,"[53e99cf5b7602d97025ad76a, 53e9984bb7602d97020...","[{'_id': '53f464b5dabfaeb22f536448', 'name': '...",2014


# Creating dataset

Dataset consist from tuples: `(entity1, relation, entity2)`, where  relation $\in$ {`REFERENCE`, `AUTHOR`, `COAUTHOR`}

In [71]:
dataset = []

coauthors = set()  # to exclude repeated coathors
for _, article in tqdm(articles.iloc[:N].iterrows(), total=N):
    dataset.extend([[article._id, 'REFERENCE', ref, article.year] for ref in article.references])
    
    authors = sorted(author['_id'] for author in article.authors)
    dataset.extend([[author, 'AUTHOR', article._id, article.year] for author in authors])
    
    curr_coauthors = set(filter(lambda p: p not in coauthors, combinations(authors, 2)))
    dataset.extend([[author1, 'COAUTHOR', author2, article.year] for (author1, author2) in curr_coauthors])
    
    coauthors |= curr_coauthors

print(len(dataset))
dataset = np.array(dataset)
print(Counter(dataset[:, 1]))

  0%|          | 0/100000 [00:00<?, ?it/s]

3163
Counter({'REFERENCE': 1849, 'COAUTHOR': 755, 'AUTHOR': 559})


In [47]:
print(dataset[:20])

[['56d85787dabfae2eee3492a5' 'REFERENCE' '55323d3f45cec66b6f9dde71'
  '2015']
 ['56d85787dabfae2eee3492a5' 'REFERENCE' '5736954c6e3b12023e4789f9'
  '2015']
 ['56d85787dabfae2eee3492a5' 'REFERENCE' '53e99bb1b7602d9702455f09'
  '2015']
 ['56d85787dabfae2eee3492a5' 'REFERENCE' '53e9a806b7602d970314667b'
  '2015']
 ['56d85787dabfae2eee3492a5' 'REFERENCE' '53e9b109b7602d9703b898e2'
  '2015']
 ['56d85787dabfae2eee3492a5' 'REFERENCE' '5736954c6e3b12023e4789a0'
  '2015']
 ['56d85787dabfae2eee3492a5' 'REFERENCE' '53e9a46bb7602d9702d885f6'
  '2015']
 ['54068615dabfae8faa60c7c4' 'AUTHOR' '56d85787dabfae2eee3492a5' '2015']
 ['56d8578edabfae2eee34c50a' 'REFERENCE' '53e9990cb7602d9702146f11'
  '2015']
 ['56d8578edabfae2eee34c50a' 'REFERENCE' '53e9a62eb7602d9702f5b6a0'
  '2015']
 ['56d8578edabfae2eee34c50a' 'REFERENCE' '53e9bb23b7602d970475e3b7'
  '2015']
 ['56d8578edabfae2eee34c50a' 'REFERENCE' '573697d46e3b12023e6b3582'
  '2015']
 ['53f452cfdabfaee02ad4b291' 'AUTHOR' '56d8578edabfae2eee34c50a' '201

In [72]:
author_weights = dict()
for item in dataset:
    if item[1] == 'AUTHOR':
        if item[0] not in author_weights:
            author_weights[item[0]] = 1
        else:    
            author_weights[item[0]]+=1

In [73]:
nodes = list(author_weights.keys())
weights = list(author_weights.values())
print(nodes[0:10])
print(weights[0:10])

['54068615dabfae8faa60c7c4', '53f452cfdabfaee02ad4b291', '53f464ffdabfaedf436407df', '54329691dabfaeb4c6a9080f', '562d1b2045cedb3398d54413', '5606653745cedb3396868235', '53f438a7dabfaec09f19263c', '53f44c92dabfaefedbb2c18b', '562d3e3145cedb3398d9be6d', '53f464b5dabfaeb22f536448']
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [74]:
graph = Network(notebook=True)
graph.add_nodes(nodes, value=weights)

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [75]:
Blues = plt.get_cmap('Blues')
def make_year_to_color(year):
    ratio = (year - min_year) / (max_year  - min_year + 1)
    return Blues(ratio)


In [77]:
for item in dataset:
    if item[1] == 'REFERENCE' and item[0] in author_weights and item[2] in author_weights:
        graph.add_edge(item[0], item[2], color=make_year_to_color(int(item[3])))

In [78]:
graph.show('grpah_200_articles.html')